In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime
import sys
import sparkStructuredStreaming
import os

### Necessary to stream from Kafka topic

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 pyspark-shell'

In [3]:
#"127.0.0.1:9092" (local) //"10.0.0.8:9092" (BACC)
bootstrap = "127.0.0.1:9092"
hdfs_path = "hdfs://0.0.0.0:19000"
output_dir = "iex/quotes"

### Initialize Spark Session + Kafkastream

In [4]:
# udf to convert epoch time to spark TimestampType
get_timestamp = udf(lambda x : datetime.datetime.fromtimestamp(x/ 1000.0).strftime("%Y-%m-%d %H:%M:%S"))

In [5]:
spark = SparkSession \
            .builder \
            .appName("KafkaIEXStructuredStreaming") \
            .master("local[*]") \
            .getOrCreate()

In [6]:
sss = sparkStructuredStreaming.kafka_spark_stream(bootstrap)

parsedDF = sss.stream_quotes(spark)       

selectDF = parsedDF \
        .select(explode(array("quote_data")))\
        .select(get_timestamp("col.latestUpdate").cast("timestamp").alias("timestamp"),"col.latestPrice")\
        .withWatermark("timestamp", "15 hours")

### Some functions for analysis

In [7]:
def time_chart(df,interval):
    # use df with "timestamp", "latestPrice", "Watermark"
    # get open, high, low prices for each time interval
    interval_values = df.groupBy(
        window(df.timestamp, interval))\
        .agg(max("latestPrice").alias("high"),\
            min("latestPrice").alias("low"),\
            min("timestamp").alias("open_time"))\
        .select("window.start","window.end","high","low","open_time")\
        .withWatermark("start", interval)
    
    # join to get opening price from opening time
    chart = interval_values.join(df,interval_values.open_time == df.timestamp, "left")\
        .drop("open_time","timestamp")\
        .withColumnRenamed("latestPrice","open")
        
    return chart

In [8]:
def moving_average(spark, df, update, interval):
    # simple moving average for the interval "interval"
    
    windowdf = df.select(window(df.timestamp, interval, update), df.latestPrice)
    
    windowdf.createOrReplaceTempView("windowdf_sql")
    
    sma = spark.sql("""SELECT windowdf_sql.window AS time, avg(windowdf_sql.latestPrice) AS average
                    FROM windowdf_sql
                    Group BY windowdf_sql.window
                    """)   
    return sma

### Playground

In [9]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
%matplotlib inline

In [11]:
def plot_stream(df, epoch_id):
    df = df.orderBy("timestamp")
    df_pd = df.toPandas()
    clear_output(wait=True)
    df_pd

In [ ]:
average = moving_average(spark, selectDF, "1 minutes" ,"8 minutes")
selectDF\
    .writeStream\
    .outputMode("append")\
    .trigger(processingTime = "60 seconds")\
    .foreachBatch(plot_stream)\
    .start()\
    .awaitTermination()